In [158]:
!pip install pandas
!pip install seaborn

In [159]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns

In [160]:
### Loading the Dataset
df=pd.read_csv('./data/Oakland_Auto_Theft.csv')
df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152727 entries, 0 to 152726
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   CrimeType    149889 non-null  object
 1   DateTime     152727 non-null  object
 2   CaseNumber   152727 non-null  object
 3   Description  152727 non-null  object
 4   PoliceBeat   152196 non-null  object
 5   Address      152677 non-null  object
 6   City         152727 non-null  object
 7   State        152727 non-null  object
 8   Location     152726 non-null  object
dtypes: object(9)
memory usage: 10.5+ MB


In [161]:
### Create a unique CrimeType in pair key:value dictionary 
### For later integer encoding
type(df['CrimeType'].unique())
CrimeType_map= dict(enumerate(df['CrimeType'].unique(), 1))
print(CrimeType_map)

{1: 'TOWED VEHICLE', 2: 'ROBBERY', 3: 'STOLEN VEHICLE', 4: 'OTHER', 5: 'RECOVERED O/S STOLEN', 6: nan, 7: 'THEFT', 8: 'GRAND THEFT', 9: 'NARCOTICS', 10: 'ARSON', 11: 'BURG - AUTO', 12: 'RECOVERED VEHICLE - OAKLAND STOLEN', 13: 'BURG - RESIDENTIAL', 14: 'STOLEN AND RECOVERED VEHICLE', 15: 'DOMESTIC VIOLENCE', 16: 'BURG - COMMERCIAL', 17: 'THREATS', 18: 'OUTSIDE AGENCY INCIDENT', 19: 'PETTY THEFT', 20: 'WEAPONS', 21: 'DISORDERLY CONDUCT', 22: 'INCIDENT TYPE', 23: 'EMBEZZLEMENT', 24: 'FORCIBLE RAPE', 25: 'BURG - OTHER', 26: 'FORGERY & COUNTERFEITING', 27: 'FELONY ASSAULT', 28: 'FELONY WARRANT', 29: 'HOMICIDE', 30: 'KIDNAPPING', 31: 'MISSING', 32: 'FRAUD', 33: 'MISDEMEANOR ASSAULT', 34: 'POSSESSION - STOLEN PROPERTY', 35: 'MISCELLANEOUS TRAFFIC CRIME', 36: 'RECOVERED VEHICLE-OLD SYSTEM STOLEN', 37: 'DUI', 38: 'VANDALISM', 39: 'OTHER SEX OFFENSES', 40: 'RUNAWAY', 41: 'MISDEMEANOR WARRANT', 42: 'PROSTITUTION', 43: 'DRUNKENNESS', 44: 'CURFEW & LOITERING', 45: 'PRE-LRMS/MISSING IN SYSTEM CONVE

In [162]:
### Police Beat Unique values
df['PoliceBeat'].unique()

array(['31Y', '06X', '16Y', '32Y', '07X', '27Y', '10Y', '31Z', '01X',
       '15X', '23X', '08X', '12X', '29X', '02Y', '34X', '77X', '05X',
       '14X', '32X', '33X', '10X', '27X', '26X', '24Y', '03Y', '35Y',
       '21Y', '14Y', '13Z', '35X', '18Y', '17X', '26Y', '13Y', '19X',
       '22Y', '25X', '02X', '30X', '20X', '30Y', '03X', '99X', '22X',
       '24X', '21X', '11X', '25Y', '05Y', '04X', '31X', '17Y', '13X',
       '12Y', '09X', '33Y', '28X', '16X', '18X', '35', '327', '32', 'PCW',
       '4', '5', '28', 'PDT2', nan, '17 Y', '6X', '02 X', '1', '94603',
       '3X', '2Y', '7X', '30 X', '8X', '17', '9X', '27 Y', '29', '1X',
       '4X', '3Y', '7', '06 X', '31 Z', '04Y', '1Y', '0', '22', '2X', 'X',
       '34Y', '5Y', '88X', 'P3', '30', '07', '34', '27', '21', '26', '08',
       '33', '24', '10', '12', '31', '20', '18', '03', '25', '5X', '16',
       '04', '02', '06', '09', '05', '19', '01', '8', '11', '23', '6',
       '13', '14', '9', '15', 'CCD2'], dtype=object)

In [163]:
### Convert string DateTime to DateTime type object
df['DateTime'] = pd.to_datetime(df['DateTime'])

C:\Windows\Temp\ipykernel_2744\3175685190.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['DateTime'])


In [164]:

### Split Day, Month, Year, Hour in DateTime object
df['Day'] = df['DateTime'].dt.day
df['Month']= df['DateTime'].dt.month
df['Year']= df['DateTime'].dt.year
df['Hour']= df['DateTime'].dt.hour
df['DayOfWeek']= df['DateTime'].dt.weekday
df["DayOfWeekName"]= df['DateTime'].dt.strftime('%A')

In [165]:
### Dropping non-important features
df.drop(['CaseNumber', 'DateTime', 'Description', 'City', 'State'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152727 entries, 0 to 152726
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   CrimeType      149889 non-null  object
 1   PoliceBeat     152196 non-null  object
 2   Address        152677 non-null  object
 3   Location       152726 non-null  object
 4   Day            152727 non-null  int32 
 5   Month          152727 non-null  int32 
 6   Year           152727 non-null  int32 
 7   Hour           152727 non-null  int32 
 8   DayOfWeek      152727 non-null  int32 
 9   DayOfWeekName  152727 non-null  object
dtypes: int32(5), object(5)
memory usage: 8.7+ MB


In [166]:
df.head(5)

,CrimeType,PoliceBeat,Address,Location,Day,Month,Year,Hour,DayOfWeek,DayOfWeekName
0,TOWED VEHICLE,31Y,9609 MADDUX DR,POINT (-122.186243032 37.736909007),30,9,2022,23,4,Friday
1,ROBBERY,06X,2420 SAN PABLO AV,POINT (-122.274354017 37.81511099),5,10,2022,3,2,Wednesday
2,STOLEN VEHICLE,16Y,1123 EL CENTRO AV,POINT (-122.219454982 37.810370982),27,11,2013,11,2,Wednesday
3,OTHER,32Y,2374 108TH AV,POINT (-122.153223028 37.740880981),19,12,2013,23,3,Thursday
4,STOLEN VEHICLE,07X,3129 SAN PABLO AV,POINT (-122.277448029 37.822463008),28,1,2014,11,1,Tuesday


In [167]:
### Split the location to X, Y coordinates
df[['Point','X', 'Y']] = df['Location'].str.split(' ', expand= True)

In [168]:
### Remove "(" ")" from X and Y columns
### Drop unnecessary 'Point' columns
df['X']= df['X'].str.slice(1)
df['Y'] = df['Y'].str.rstrip(')')
df.drop('Point', axis = 1, inplace= True)

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152727 entries, 0 to 152726
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   CrimeType      149889 non-null  object
 1   PoliceBeat     152196 non-null  object
 2   Address        152677 non-null  object
 3   Location       152726 non-null  object
 4   Day            152727 non-null  int32 
 5   Month          152727 non-null  int32 
 6   Year           152727 non-null  int32 
 7   Hour           152727 non-null  int32 
 8   DayOfWeek      152727 non-null  int32 
 9   DayOfWeekName  152727 non-null  object
 10  X              152726 non-null  object
 11  Y              152726 non-null  object
dtypes: int32(5), object(7)
memory usage: 11.1+ MB


In [170]:
df.head(5)

,CrimeType,PoliceBeat,Address,Location,Day,Month,Year,Hour,DayOfWeek,DayOfWeekName,X,Y
0,TOWED VEHICLE,31Y,9609 MADDUX DR,POINT (-122.186243032 37.736909007),30,9,2022,23,4,Friday,-122.186243032,37.736909007
1,ROBBERY,06X,2420 SAN PABLO AV,POINT (-122.274354017 37.81511099),5,10,2022,3,2,Wednesday,-122.274354017,37.81511099
2,STOLEN VEHICLE,16Y,1123 EL CENTRO AV,POINT (-122.219454982 37.810370982),27,11,2013,11,2,Wednesday,-122.219454982,37.810370982
3,OTHER,32Y,2374 108TH AV,POINT (-122.153223028 37.740880981),19,12,2013,23,3,Thursday,-122.153223028,37.740880981
4,STOLEN VEHICLE,07X,3129 SAN PABLO AV,POINT (-122.277448029 37.822463008),28,1,2014,11,1,Tuesday,-122.277448029,37.822463008


In [174]:
### Convert from String to FLoat
df['X'] = df['X'].astype(float)
df['Y'] = df['Y'].astype(float)

In [176]:
### Remove Location column
df.drop('Location', axis = 1, inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152727 entries, 0 to 152726
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   CrimeType      149889 non-null  object 
 1   PoliceBeat     152196 non-null  object 
 2   Address        152677 non-null  object 
 3   Day            152727 non-null  int32  
 4   Month          152727 non-null  int32  
 5   Year           152727 non-null  int32  
 6   Hour           152727 non-null  int32  
 7   DayOfWeek      152727 non-null  int32  
 8   DayOfWeekName  152727 non-null  object 
 9   X              152726 non-null  float64
 10  Y              152726 non-null  float64
dtypes: float64(2), int32(5), object(4)
memory usage: 9.9+ MB


In [177]:
df.head(5)

,CrimeType,PoliceBeat,Address,Day,Month,Year,Hour,DayOfWeek,DayOfWeekName,X,Y
0,TOWED VEHICLE,31Y,9609 MADDUX DR,30,9,2022,23,4,Friday,-122.186243,37.736909
1,ROBBERY,06X,2420 SAN PABLO AV,5,10,2022,3,2,Wednesday,-122.274354,37.815111
2,STOLEN VEHICLE,16Y,1123 EL CENTRO AV,27,11,2013,11,2,Wednesday,-122.219455,37.810371
3,OTHER,32Y,2374 108TH AV,19,12,2013,23,3,Thursday,-122.153223,37.740881
4,STOLEN VEHICLE,07X,3129 SAN PABLO AV,28,1,2014,11,1,Tuesday,-122.277448,37.822463
